In [65]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import json

In [68]:
pwd

'/Users/christinedekock/wp_internship/data/formatted_data'

In [115]:
with open("/Users/christinedekock/wp_internship/data/data_export_1402/collaborationPatterns-autobiography-meta-info.csv",'r') as my_file:
    text = [k.split('\t') for k in my_file.readlines()]
    
text = [i[:16] for i in text if len(i) >= 16]

In [116]:
blp = pd.read_csv('/Users/christinedekock/wp_internship/data/namespaces.csv')
blp = blp.query('page_namespace in (0,1)')

In [117]:
text_blp = [i for i in tqdm(text[1:]) if int(i[2]) in blp.page_id.values]

100%|██████████| 3689444/3689444 [1:03:41<00:00, 965.51it/s]


In [118]:
revisions = pd.DataFrame(text_blp, columns=text[0])
revisions['page_namespace'] = revisions['page_namespace'].astype(int)
revisions = revisions.query('page_namespace in (0,1)')

In [119]:
def get_edits_pre_tag(data):
    ix = data['has_template'].idxmax()

    return data.loc[:ix]

In [120]:
revisions = revisions.sort_values(by=['page_id','revision_id'], ascending=True)
revisions['has_template'] = revisions['has_template'].fillna(0).astype(int)
pre_edit = revisions.groupby('page_id').apply(get_edits_pre_tag)
pre_edit.reset_index(drop=True, inplace=True)

In [121]:
revisions.groupby(['page_namespace'])['page_id'].count()

page_namespace
0    631111
1      1659
Name: page_id, dtype: int64

In [122]:
revisions.groupby(['page_namespace'])['page_id'].nunique()

page_namespace
0    3643
1      17
Name: page_id, dtype: int64

In [ ]:
# persist

In [123]:
revisions.to_csv('/Users/christinedekock/wp_internship/data/formatted_data/autobiography.csv',index=False)

In [124]:
with open('/Users/christinedekock/wp_internship/data/formatted_data/autobiography.json','w') as f:
    json.dump(revisions.to_dict(orient='records'),f)